In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

from sklearn import metrics
from sklearn.metrics import accuracy_score

### Carregando a Base e Definindo as amostras de Treino e Teste

In [ ]:
def carregarBase(base, isTodosAtributos, colunasRemovidas):
    covidData = pd.read_excel(base)
    if(isTodosAtributos == False):
        covidData = covidData.drop(columns = colunasRemovidas)
    return covidData

def criarTreinamentoTeste(tamanhoTeste, baseDeDados, alvo):
    return train_test_split(baseDeDados.drop(alvo, axis=1), baseDeDados[alvo], 
                            test_size = tamanhoTeste, random_state = 42)

### Criando o modelo a partir do algoritmo

In [ ]:
def criarModelo(modelo, hiperparametros):
    if(modelo == "AB"):
        #classificador = AdaBoostClassifier(hiperparametros)
        classificador = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                                           n_estimators=50, random_state=410)
        return classificador
    
    if(modelo == "RF"):
        #classificador = RandomForestClassifier(hiperparametros)
        classificador = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, 
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=7374, verbose=0,
                       warm_start=False)
        return classificador
    
    if(modelo == "LR"):
        #classificador = LogisticRegression(hiperparametros)
        classificador = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2576, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
        return classificador
    
    if(modelo == "GB"):
        #classificador = GradientBoostingClassifier(hiperparametros)
        classificador = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, 
                           random_state=2333, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
        return classificador

### Calculando a acurácia, precisão, recall e f1-score

In [ ]:
def calcularMetricas(x_train, x_test, y_train, y_test, baseDeDados, classificador, alvos):
    metricas = {}
    
    # Fit nos dados
    classificador.fit(x_train, y_train)

    #Realizando a predição
    resultadoPredicao = classificador.predict(x_test)

    # Verificando a acurácia
    metricas['Acurácia'] = __transformarEmPorcentagem__(accuracy_score(y_test, resultadoPredicao))
    
    # Criando um dicionaŕio a partir das métricas
    metricasValores = metrics.classification_report(y_test, resultadoPredicao, output_dict=True)
    df = pd.DataFrame(metricasValores).transpose()

    for classe in alvos:
        metricas["Precisão (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['precision'])
        metricas["Recall (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['recall'])
        metricas["F1-Score (" + str(classe) + ")"] = __transformarEmPorcentagem__(df.loc[str(classe)]['f1-score'])
    
    return metricas

### Calculando os K-folds

In [ ]:
def calcularKfolds(dados, alvo, classificador):
    kfolds = {}
    kfolds["3 Kfolds"] = __transformarEmPorcentagem__(__calcularKfoldIndividual__(3, dados, alvo, classificador))
    kfolds["5 Kfolds"] = __transformarEmPorcentagem__(__calcularKfoldIndividual__(5, dados, alvo, classificador))
    return kfolds
    
def __calcularKfoldIndividual__(kfolds, baseDeDados, alvo, classificador):
    data, target = baseDeDados.drop(columns = [alvo]), baseDeDados[alvo]
    scores = cross_val_score(classificador, data, target, cv = kfolds, scoring = "accuracy") 
    return scores.mean()

### Utilitarios para transformação de dados

In [ ]:
def __transformarEmPorcentagem__(valor):
    porcentagem = str(round(valor * 100, 3)) + "%"
    return porcentagem

def converterMapas(baseMap, metricasMap, kfoldsMap):
    baseMap.update(metricasMap)
    baseMap.update(kfoldsMap)
    return pd.DataFrame([baseMap])

def juntarBases (base_1, base_2):
    return pd.concat([base_1, base_2])

### Avaliar os Modelos

In [ ]:
def testarModelos(bases, alvo, colunasRemovidas, modelos, alvos):

    desempenhoDosModelos = pd.DataFrame()
    for nomeBase in bases.keys():
        covidData = carregarBase(bases[nomeBase], False, colunasRemovidas)
        x_train, x_test, y_train, y_test = criarTreinamentoTeste(0.3, covidData, alvo)

        for nomeModelo in modelos.keys():
            
            #modelo = criarModelo(nomeModelo, modelos[nomeModelo])
            modelo = modelos[nomeModelo]
            baseMap = {'Base': nomeBase, 'Modelo': nomeModelo, 'Hiperparâmetros': modelo.get_params()}
            
            metricasMap = calcularMetricas(x_train, x_test, y_train, y_test, covidData, modelo, alvos)
            kfoldsMap = calcularKfolds(covidData, alvo, modelo)
            desempenhoDosModelos = juntarBases (desempenhoDosModelos, converterMapas(baseMap, metricasMap, kfoldsMap))

    return desempenhoDosModelos

### Salvando o dataframe em uma planilha

In [ ]:
def salvarBaseDeDados(baseDeDados, arquivo):
    write = pd.ExcelWriter(arquivo)
    baseDeDados.to_excel(write, 'dados', index = False)
    write.save()

### Juntando os lotes da Primeira Análise

In [ ]:
def juntarLotes(proporcao, reduzDados):
    spLote1 = pd.read_excel("SP/sp-l1-" + proporcao + "-2022.xlsx")
    spLote2 = pd.read_excel("SP/sp-l2-" + proporcao + "-2022.xlsx")
    spLote3 = pd.read_excel("SP/sp-l3-" + proporcao + "-2022.xlsx")
    spLote4 = pd.read_excel("SP/sp-l4-" + proporcao + "-2022.xlsx")
    spLote5 = pd.read_excel("SP/sp-l5-" + proporcao + "-2022.xlsx")

    # Unindo os lotes
    covidMerged = pd.concat([spLote1, spLote2, spLote3, spLote4, spLote5])

    # Resetando os índices
    covidMerged.reset_index(inplace=True, drop = True)
    
    if(reduzDados):
        colunasRemovidas = [ 'disturbiosOlfativos', 'disturbiosGustatorios', 'puerpera', 'fragilidadeImuno', 
                    'gestante', 'obesidade']
        covidMerged = covidMerged.drop(columns = colunasRemovidas)
    
    return covidMerged